In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import glm
from scipy.stats import chisquare, poisson
import matplotlib.pyplot as plt
import scipy.stats as stats

from mickelonius_stats.hypothesis_testing.count.data.generate_poisson_data import generate_poisson_data

RATE1 = 10
RATE2 = 5
N_LG = 1000
N_SM = 25

# Example data
sample1_lg = generate_poisson_data(RATE1, N_LG)
sample2_lg = generate_poisson_data(RATE2, N_LG)

# Combine the samples
counts = np.concatenate([sample1_lg, sample2_lg])
groups = np.concatenate([np.zeros_like(sample1_lg), np.ones_like(sample2_lg)])

# Create a DataFrame
data = pd.DataFrame({'counts': counts, 'group': groups})

# Test if data look they are poisson distributed
## 1. Is mean ~ variance?

In [ ]:
mean1, variance1 = np.mean(sample1_lg), np.var(sample1_lg, ddof=1)
mean2, variance2 = np.mean(sample2_lg), np.var(sample2_lg, ddof=1)
print('\nIs mean ~ variance, as is the case for a Poisson Distributed RV?')
print('#' * 70)
print(f"# Sample 1: mean={mean1:.2f}, variance={variance1:.2f}")
print(f"# Sample 2: mean={mean2:.2f}, variance={variance2:.2f}")
print('#' * 70)

## 2. Visual Inspection, Plot histogram of the data and overlay the Poisson distribution

In [ ]:
x1 = np.arange(min(sample1_lg), max(sample1_lg) + 1)
x2 = np.arange(min(sample2_lg), max(sample2_lg) + 1)
plt.hist(sample1_lg, bins=x1, alpha=0.75, color='blue', density=True, label='Observed, Sample 1')
plt.hist(sample2_lg, bins=x2, alpha=0.75, color='green', density=True, label='Observed, Sample 2')

# Overlay the Poisson distribution
poisson_pmf1 = stats.poisson.pmf(x1, mu=mean1)
plt.plot(x1+0.5, poisson_pmf1, 'bo', ms=8, label='Expected Poisson, Sample 1')  #, color='blue')
plt.vlines(x1+0.5, 0, poisson_pmf1, color='blue', lw=1)
poisson_pmf2 = stats.poisson.pmf(x2, mu=mean2)
plt.plot(x2+0.5, poisson_pmf2, 'go', ms=8, label='Expected Poisson, Sample 2')  #, color='green')
plt.vlines(x2+0.5, 0, poisson_pmf2, color='green', lw=1)
plt.title('Histograms and Poisson PMFs')
plt.xlabel('Counts')
plt.ylabel('Frequency')
plt.legend()
plt.show()

## 3. QQ Plot

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 6))
fig.suptitle('Q-Q Plots for Poisson-Distributed Data', fontsize=16)
stats.probplot(sample1_lg, dist="poisson", sparams=(mean1,), plot=axs[0])
axs[0].set_title(f"Rate = {RATE1}")
stats.probplot(sample2_lg, dist="poisson", sparams=(mean2,), plot=axs[1])
axs[1].set_title(f"Rate = {RATE2}")
plt.tight_layout()
plt.show()

## 4. Chi-Square Goodness-of-Fit tests
Note that p-values well above (not below) significance level are desired as the null hypothesis is that the data does fit the ditribution, therefore we DON'T want evidence that rejects the null hypothesis.

In [ ]:
samples = [sample1_lg, sample2_lg]
min_max = [(float(min(s)), float(max(s))) for s in samples]
means = [np.mean(s) for s in samples]
print('\nDo samples fit Poisson Distribution?')
print('#' * 70)
for i, (min_val, max_val) in enumerate(min_max):
    values = np.arange(min_val, max_val + 1)
    observed_freq = np.array([(samples[i] == value).sum() for value in values])
    expected_freq = len(samples[i]) * poisson.pmf(values, mu=means[i])
    # expected_freq sums need to almost exactly equal for chi-sq test
    expected_freq = expected_freq * (observed_freq.sum() / expected_freq.sum())
    chi2_stat, p_value = chisquare(f_obs=observed_freq, f_exp=expected_freq)
    print(f"Chi-Square Statistic for sample {i + 1}: {chi2_stat:.2f}, P-Value: {p_value:.2f}")
print('#' * 70)

# Use Poisson GLM to fit counts to group/sample

In [ ]:
model = glm('counts ~ group', data, family=sm.families.Poisson()).fit()
print(model.summary())

### Calculate the rate and CI

In [ ]:
intercept_coef = model.params['Intercept']
group_coef = model.params['group']

conf_int = model.conf_int()
conf_int_exp = np.exp(conf_int)
conf_int_intercept = conf_int_exp.loc['Intercept']
conf_int_group = conf_int_exp.loc['group']

rate_est = float(np.exp(intercept_coef))
rate_est2 = np.exp(intercept_coef + group_coef)
ci1 = [float(c) for c in conf_int_intercept]
ci2 = [float(np.exp(intercept_coef + conf_int.loc['group'][i])) for i in [0,1]]

print('\nPossion GLM fit on groups denoting sample set')
print('#' * 70)
print(f'# Estimated rate for sample 1: {rate_est:.2f} CI=[{ci1[0]:.2f},{ci1[1]:.2f}]')
print(f'# Estimated rate for sample 2: {rate_est2:.2f} CI=[{ci2[0]:.2f},{ci2[1]:.2f}]')
print('#' * 70)